<a href="https://colab.research.google.com/github/Tomiwa29/tomiwa/blob/master/Suspicious%20transaction_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Introduction

Scenario:


Goal:
Detect suspicious transactions using Ml



# **Importing necessary liberies**

In [ ]:
! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from pandas_profiling import ProfileReport


In [ ]:
# Reading the Dataset

df = pd.read_csv("/content/PS_20174392719_1491204439457_log.csv")

df.head()

In [ ]:
df.tail()

In [ ]:
df.describe()
# is a Pandas method that computes a summary of statistics for each column in a DataFrame. It returns a new DataFrame containing the following statistics:
#count: the number of non-null values in each column mean: the mean of each column std: the standard deviation of each column min: the minimum value in each column 50%: the 50th percentile (median) of each column 75%: the 75th percentile of each column max: the maximum value in each column
#This method can be useful for gaining a quick understanding of the distribution of values in each column of a DataFrame, and for detecting potential issues such as missing or outlier values.



In [ ]:
df.dtypes

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.isnull().values.any()

In [ ]:
list(df.loc[df.isFraud == 0].type.drop_duplicates().values)

In [ ]:
list(df.loc[df.isFraud == 1].type.drop_duplicates().values)

In [ ]:
dft = df.loc[(df.isFraud == 1) & (df.type == 'TRANSFER')]
dfc = df.loc[(df.isFraud == 1) & (df.type == 'CASH_OUT')]

In [ ]:
#Number of fraudulent TRANSFER
len(dft)

In [ ]:
#Number of fraudulent CASH_OUT
len(dfc)

In [ ]:
def prep_data(df: pd.DataFrame) -> (np.ndarray, np.ndarray):
    """
    Convert the DataFrame into two variable
    X: data columns (V1 - V28)
    y: lable column
    """
    X = df.iloc[:, 2:30].values
    y = df.Class.values
    return X, y

In [ ]:
df.profile_report()

In [ ]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()

df['nameOrig']=le.fit_transform(df['nameOrig'])
df['type']=le.fit_transform(df['type'])
df['nameDest']=le.fit_transform(df['nameDest'])

df.head(9)
# 3 is payment 4 is TRANSFER 1 is CASHOUT

We check if their is a good proportion between our positive and negative binary predictor.

In [ ]:
df['isFraud'].value_counts()
#This means that there are 137993 rows where the value in the isFraud column is 0 and 122 rows where the value is 1.

# Create a histogram of transaction amounts
plt.hist(df['transaction_amount'], bins=50)
plt.xlabel('Transaction Amount')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Create a histogram of transaction amounts
plt.hist(df['amount'], bins=20)
plt.xlabel('amount')
plt.ylabel('frequency')
plt.show()

In [ ]:
# Create a scatterplot of transaction amounts by time
sns.scatterplot(x='type', y='amount', hue='isFraud', data=df)
plt.show()

### Correlation Matrix
Correlation Matrix lets you see correlations between all variable. How highly or weak each variable is related to each other.

In [ ]:
fraud_people = df[df['isFraud']==1]
legit_people = df[df['isFraud']==0]

In [ ]:
fraud_people.shape

In [ ]:
legit_people.shape

In [ ]:
fraud_people['amount'].describe()

In [ ]:
graph, (plot1, plot2) = plt.subplots(2,1,sharex = True)
graph.suptitle("Average amount per class")
bins = 70

plot1.hist(fraud_people['amount'], bins = bins)
plot1.set_title('Fraud Amount')

plot2.hist(legit_people['amount'], bins = bins)
plot2.set_title('Legit Amount')

plt.xlabel('amount')
plt.ylabel('Number of Transactions')
plt.yscale('log')
plt.show();

In [ ]:
columns = df.columns.tolist()
columns = [var for var in columns if var not in ["isFraud"]] #independent
target = "isFraud"
x = df[columns]
y = df[target] #dependent

In [ ]:
x['type'].replace(['CASH_IN','CASH_OUT','DEBIT','PAYMENT','TRANSFER'],['1','2','3','4','5',], inplace=True)

In [ ]:
x.drop(['nameOrig','nameDest','isFlaggedFraud'],axis=1, inplace=True)

In [ ]:
y.head()

In [ ]:
x.head()

By comparing positve and negative patients, we can see there are vast differences in means for many of our 13 features.

In [ ]:
# Add random noise to the features
noise = np.random.normal(loc=0, scale=0.1, size=x.shape)  # Adjust the noise scale as needed
x_noisy = x + noise

# Introduce outliers
outliers = np.random.uniform(low=-10, high=10, size=(10, x.shape[1]))  # Generate 10 outlier instances
x_outliers = np.concatenate((x, outliers), axis=0)
y_outliers = np.concatenate((y, np.ones((10,))), axis=0)


## Prepare Data for Modeling
To prepare data for modeling, remember ASN (Assign, Split, Normalize)

In [ ]:
print(df.columns)


In [ ]:
df.dropna(subset=['isFraud'], inplace=True)


In [ ]:
missing_values = y.isnull().sum()
print("Missing values in y:", missing_values)


In [ ]:
from sklearn.impute import SimpleImputer

# Impute missing target value using mean imputation
imputer = SimpleImputer(strategy='mean')
y_imputed = imputer.fit_transform(y.values.reshape(-1, 1))


In [ ]:
print(y.describe())


In [ ]:
print(y[y.isnull()])


In [ ]:
from sklearn.impute import SimpleImputer

# Initialize SimpleImputer
imputer = SimpleImputer(strategy='mean')

# Impute missing values in y
y_imputed = imputer.fit_transform(y.values.reshape(-1, 1))

# Convert back to a pandas Series
y_imputed = pd.Series(y_imputed.flatten(), index=y.index)


In [ ]:

from sklearn.preprocessing import Binarizer

# Initialize Binarizer
binarizer = Binarizer(threshold=0.5)  # Adjust the threshold as needed

# Binarize the target variable
y_binarized = binarizer.fit_transform(y_imputed.values.reshape(-1, 1))

# Convert back to a pandas Series
y_binarized = pd.Series(y_binarized.flatten(), index=y.index)


In [ ]:
from imblearn.over_sampling import RandomOverSampler

# Initialize RandomOverSampler
resampler = RandomOverSampler()

# Resample the data
x_resampled, y_resampled = resampler.fit_resample(x, y_binarized)


In [ ]:

from imblearn.over_sampling import RandomOverSampler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Initialize RandomOverSampler
resampler = RandomOverSampler()

# Resample the data
x_resampled, y_resampled = resampler.fit_resample(x, y_binarized)

# Initialize SimpleImputer
imputer = SimpleImputer(strategy='mean')

# Impute missing values in x_resampled
x_resampled_imputed = imputer.fit_transform(x_resampled)

# Split the resampled and imputed data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_resampled_imputed, y_resampled, test_size=0.2, random_state=42)

# Train a Random Forest classifier on the training set
model = RandomForestClassifier(random_state=42)
model.fit(x_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(x_test)

# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Create a heatmap for visualization
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')

# Add labels, title, and axis ticks
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')

# Save the figure as an image
plt.savefig('confusion_matrix.png')

# Show the plot
plt.show()

# Evaluate the performance of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

from sklearn.metrics import precision_score, recall_score, f1_score, mean_absolute_error, mean_squared_error, r2_score

# Calculate precision, recall, and F1-score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Calculate R^2 score
r2 = r2_score(y_test, y_pred)
print("R^2 Score:", r2)



In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# Generate predicted probabilities for the positive class
y_pred_proba_rf = model.predict(x_test)
y_pred_proba_rf = y_pred_proba_rf.flatten()  # Flatten the predictions to a 1D array

# Calculate false positive rate, true positive rate, and thresholds
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, y_pred_proba_rf)

# Calculate the AUC score
auc_rf = roc_auc_score(y_test, y_pred_proba_rf)

# Plot the ROC curve
plt.plot(fpr_rf, tpr_rf, label='Random Forest (AUC = {:.2f})'.format(auc_rf))
plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line for random guessing
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic - Random Forest')
plt.legend()
plt.show()


In [ ]:
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Initialize and fit K-means clustering algorithm
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(x_resampled_imputed)

# Predict cluster labels
cluster_labels = kmeans.predict(x_resampled_imputed)

# If ground truth labels are not available, calculate accuracy using an alternative method
if 'target_variable' not in y_resampled:
    # Calculate the majority label in each cluster
    cluster_majority_label = []
    for cluster in range(2):
        cluster_samples = y_resampled[cluster_labels == cluster]
        cluster_majority_label.append(cluster_samples.mode()[0])

    # Assign the predicted label based on the majority label in each cluster
    y_pred = [cluster_majority_label[label] for label in cluster_labels]

    # Calculate accuracy
    accuracy = accuracy_score(y_resampled, y_pred)
    print("Accuracy:", accuracy)
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    from sklearn.cluster import KMeans

# Calculate the confusion matrix
cm = confusion_matrix(y_resampled, y_pred)

# Create a heatmap for visualization
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()



# Calculate precision, recall, and F1-score
precision = precision_score(y_resampled, y_pred)
recall = recall_score(y_resampled, y_pred)
f1 = f1_score(y_resampled, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


from sklearn.metrics import precision_score, recall_score, f1_score, mean_absolute_error, mean_squared_error,r2_score


# Calculate MAE
mae = mean_absolute_error(y_resampled, y_pred)
print("Mean Absolute Error:", mae)

# Calculate MSE
mse = mean_squared_error(y_resampled, y_pred)
print("Mean Squared Error:", mse)

# Calculate R^2 score
r2 = r2_score(y_resampled, y_pred)
print("R^2 Score:", r2)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
import numpy as np

# Handle missing values in x_resampled
imputer = SimpleImputer(strategy='mean')
x_resampled_imputed = imputer.fit_transform(x_resampled)

# Perform feature scaling on x_resampled_imputed
scaler = StandardScaler()
x_resampled_scaled = scaler.fit_transform(x_resampled_imputed)

# Fit K-means on the scaled data
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(x_resampled_scaled)

# Calculate the distances to the cluster centroids
distances = kmeans.transform(x_resampled_scaled)

# Use the minimum distance as the anomaly score
anomaly_scores = np.min(distances, axis=1)

# Calculate false positive rate, true positive rate, and thresholds
fpr_kmeans, tpr_kmeans, thresholds_kmeans = roc_curve(y_resampled, anomaly_scores)

# Calculate the AUC score
auc_kmeans = roc_auc_score(y_resampled, anomaly_scores)

# Plot the ROC curve
plt.plot(fpr_kmeans, tpr_kmeans, label='K-means (AUC = {:.2f})'.format(auc_kmeans))
plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line for random guessing
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic - K-means')
plt.legend()
plt.show()


In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Initialize and train the K-means clustering algorithm
kmeans = KMeans(n_clusters=3)  # Adjust the number of clusters as needed
kmeans.fit(x_resampled_imputed)

# Obtain the cluster assignments for the data points
cluster_labels = kmeans.predict(x_resampled_imputed)

# Perform further analysis or processing on the cluster assignments or centroids
# ..

# Evaluate the performance of the clustering algorithm
silhouette_score_value = silhouette_score(x_resampled_imputed, cluster_labels)
print("Silhouette Score:", silhouette_score_value)


In [ ]:
import tensorflow as tf
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, mean_absolute_error, mean_squared_error, r2_score

# Handle missing values in x_resampled
imputer = SimpleImputer(strategy='mean')
x_resampled_imputed = imputer.fit_transform(x_resampled)

# Perform feature scaling on x_resampled_imputed
scaler = StandardScaler()
x_resampled_scaled = scaler.fit_transform(x_resampled_imputed)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_resampled_scaled, y_resampled, test_size=0.2, random_state=42)

# Build the ANN model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# Evaluate the model on the testing set
loss, accuracy = model.evaluate(x_test, y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)

# Make predictions on the testing set
y_pred_ann_probs = model.predict(x_test)
y_pred_ann = (y_pred_ann_probs > 0.5).astype(int)

# Calculate precision, recall, and F1-score
precision = precision_score(y_test, y_pred_ann)
recall = recall_score(y_test, y_pred_ann)
f1 = f1_score(y_test, y_pred_ann)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred_ann)
print("Mean Absolute Error:", mae)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred_ann)
print("Mean Squared Error:", mse)

# Calculate R^2 score
r2 = r2_score(y_test, y_pred_ann)
print("R^2 Score:", r2)


In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# Generate predicted probabilities for the positive class
y_pred_proba_ann = model_ann.predict(x_test)

# Calculate false positive rate, true positive rate, and thresholds
fpr_ann, tpr_ann, thresholds_ann = roc_curve(y_test, y_pred_proba_ann)

# Calculate the AUC score
auc_ann = roc_auc_score(y_test, y_pred_proba_ann)

# Plot the ROC curve
plt.plot(fpr_ann, tpr_ann, label='ANN (AUC = {:.2f})'.format(auc_ann))
plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line for random guessing
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic - ANN')
plt.legend()
plt.show()


In [ ]:
import seaborn as sns
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

# Initialize and fit the MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(10, 10), random_state=42)
mlp.fit(x_train, y_train)

# Make predictions on the testing set
y_pred = mlp.predict(x_test)

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Create the confusion matrix plot using seaborn
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")

# Set plot labels
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")

# Show the plot
plt.show()


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

# Initialize and fit the MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(10, 10), random_state=42)
mlp.fit(x_train, y_train)

# Generate predicted probabilities for the positive class
y_pred_proba = mlp.predict_proba(x_test)[:, 1]

# Calculate precision and recall
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)

# Plot precision-recall curve
plt.plot(recall, precision)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix

# Define individual classifiers
random_forest = RandomForestClassifier(random_state=42)
ann = MLPClassifier(random_state=42)

# Define the ensemble classifier
ensemble_classifier = VotingClassifier(
    estimators=[('rf', random_forest), ('ann', ann)],
    voting='hard'
)

# Fit the ensemble classifier on the training data
ensemble_classifier.fit(x_train, y_train)

# Make predictions using the ensemble classifier
y_pred = ensemble_classifier.predict(x_test)

# Evaluate the performance of the ensemble classifier
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R^2 Score:", r2)

# Calculate and plot confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Create a confusion matrix plot
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()

tick_marks = np.arange(2)
plt.xticks(tick_marks, ['Class 0', 'Class 1'])
plt.yticks(tick_marks, ['Class 0', 'Class 1'])

plt.xlabel('Predicted Label')
plt.ylabel('True Label')

# Add labels to each cell
thresh = cm.max() / 2.0
for i, j in np.ndindex(cm.shape):
    plt.text(j, i, format(cm[i, j], 'd'),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

plt.show()


In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.cluster import KMeans
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Generate a synthetic classification dataset
X, y = make_classification(n_samples=1000, n_features=10, random_state=42)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit KMeans clustering on the training set
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X_train)

# Obtain cluster labels for the training and testing sets
X_train_clusters = kmeans.transform(X_train)
X_test_clusters = kmeans.transform(X_test)

# Concatenate the original features with the cluster distances
X_train_new = np.concatenate((X_train, X_train_clusters), axis=1)
X_test_new = np.concatenate((X_test, X_test_clusters), axis=1)

# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Fit the Random Forest classifier on the training set with new features
rf_classifier.fit(X_train_new, y_train)

# Make predictions on the testing set with new features
y_pred_rf = rf_classifier.predict(X_test_new)

# Calculate performance metrics for Random Forest
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)

# Calculate confusion matrix for Random Forest
cm_rf = confusion_matrix(y_test, y_pred_rf)

print("Random Forest Metrics:")
print("Accuracy:", accuracy_rf)
print("Precision:", precision_rf)
print("Recall:", recall_rf)
print("F1-score:", f1_rf)
print("Confusion Matrix:")
print(cm_rf)
